# Mobility Basket of Products 
##  Yearly Average Citizen Intensity Consumption calculation per country.

As a first step to estimate european citizens consumption footprint by country using Life Cycle Assessment (LCA) it is necessary to calculate the consumption intensity of the representative products considered (in general those bearing most of the env. impacts) groupped into Baskets of Products (BoP). The present script contains the functions dedicated to the extraction and harmonization of the data on the Mobility BoP products and the calculation of the corresponding consumption intensities.

The three data sources used in this case are:

- [EC Mobility and Transport Statistical Pocketbook](https://ec.europa.eu/transport/facts-fundings/statistics/pocketbook-2019_en)(PB).
- [Eurostat](https://ec.europa.eu/eurostat)(ESTAT).
- [Consumer Footprint. Basket of Products indicator on mobility](https://ec.europa.eu/jrc/en/publication/consumer-footprint-basket-products-indicator-mobility) report(Rep.).

The specific categories, datasets and tables from each of the sources used here can be found in the [ds_transport](#ds_transport) function.

As for the Mobility BoP representative products, what follows is a summary of all the representative products considered:

- **Road transport**:
  - *Passenger cars*: 16 types classified according to their fuel type, capacity and year of production.
  - *Two wheels*: 3 types classified according to their capacity and year of production.
  - *Buses*: 3 types lassified according to their weight and year of production.
- **Rail transport**: Either diesel or electric.

- **Air transport**: intra-EU, extra-EU and international flights.
 
For each of the main transport groups the following data will be retrieved:

- *Passenger cars*: million passenger km (PB), type of fuel, engine displacement and technology coefficients (ESTAT) and occupancy factors (Rep.).

- *Two wheels*: number of vehicles (ESTAT), number of vehicles (ESTAT), contribution to passenger km and occupancy factor (Rep.). Besides, the number of vehicles per category will be calculated out of the number of vehicles figures.

- *Buses*: number of vehicles (ESTAT), number of vehicles (ESTAT) and occupancy factors (Rep.). Besides, the number of vehicles per category will be calculated out of the number of vehicles figures.

- **Rail transport**: Millions of trains km (ESTAT), million passenger km (PB) and occupancy factors (Rep.). Besides the share of trains by category is calculated out of the millions of trains km figures.

- **Air transport**: Number of passengers (PB) and km per flight (Rep.).

In some cases the data retrieved from the original sources is already ready to be used, however in other cases it will have to undergo some modifications before it can be used. A third case would be those additional figures which are calculated out of the originally retrieved data, which are the following for the different transport groups:

- *Two wheels*: the number of vehicles per category will be calculated out of the number of vehicles figures.

- *Buses*: the number of vehicles per category will be calculated out of the number of vehicles figures.

- **Rail transport**: the share of trains by category is calculated out of the millions of trains km figures.

The functions in this script will fill out the spaces of a previously created frame table containing only the information about the different transport groups and subgroups names with the retrieved, modified and calcuated figures under the corresponding column. In that table the names of the columns are short forms of the original name (e.g. *tech_coeff* for technology coefficients) and its correspondance with the actual figure name can be found wihtin the [ds_transport](#ds_transport) function.


### Loading packages

In addition to Python 3.3 built-in packages and some other specific packages, such as the [*eurostat*](https://pypi.org/project/eurostat/) package, two previously developed scripts, *data_inputs* and *predownload* will have to be loaded too. In order to simplify the loading it is recommended to copy both of the into the current working dictionary, i.e. where the present script is located. Finally, within that same directory, a *input* folder has to be created and the pocketbook (e.g. *pb2019-section23.xls*) as well as the previously mentioned frame table (*mob_frame.xlsx*) copied inside it.

In [2]:
import os
import sys
import pandas as pd
from collections import Counter
import itertools
from itertools import compress
from itertools import product
import numpy as np
import eurostat as estat
import math
from scipy import interpolate

# Our scripts

import data_inputs as di
import predownload as predw

# Folders

INPUTS = "../inputs"

### Example

In [35]:
# Loading the mobility frame where the data will be integrated:
mob_file = 'mob_frame.xlsx'
file_path = os.path.join(INPUTS, mob_file)
mob_frame = pd.read_excel(file_path)

# Selecting the years interval and countries of interest:
start_year = '2010'
end_year = '2018'
countries_names = ['Austria']

# loading the Pocketbook:
file_name = 'pb2019-section23.xls'

# Running the Mobility function:
m = mobility(mob_frame, start_year, end_year, countries_names, file_name, INPUTS)


### Functions

<a id='ds_transport'></a>


In [6]:
def ds_transport(DS, col_name):
    
    """
    For the different data sets, it provides information about the way they have to be filtered, the units conversion they require,
    how they should me merged with the mobility frame, in which column will their information be displayed etc etc
    
    :param DS : data set name (EUROSTAT), sheetname (POCKETBOOK) or BoP report sheet.
    :type DS : str.
    :param col_name : name of the dictionary required.
    :type col_name : str.
    :return :  the information required, being it the units conversion factor, the filtering values etc.
    :rtype : list/dict.
    
    """
    # In each data set the values of interst for certain categories. I.e for "road_eqs_mopeds" only the rows for which
    # "mot_nrg" = "TOTAL" and "vehicle" = "MOP" will be kept. {} means no filter will be applied.
    
    filters_dict = {"road_eqs_carage":{},
                    "road_eqs_mopeds":{'mot_nrg':["TOTAL"], 'vehicle':["MOP"]},
                    "road_eqs_motorc":{},
                    "bus_coach":{},
                    "road_eqs_busmot":{},
                    "rail_tf_traveh":{'train':["TOTAL"]},
                    "road_tf_road":{"vehicle":["BUS_MCO"]},
                    "avia_panc":{'tra_meas':['PAS_BRD'], 'unit':["PAS"]},
                    "avia_paincc":{'tra_meas':['PAS_BRD_DEP'], 'unit':["PAS"], "partner" : ["EU28"]},
                    "avia_paexcc":{'tra_meas':['PAS_BRD_DEP'], 'unit':["PAS"], "partner" : ["EXT_EU"]},
                    "road_pa_mov":{},
                    "road_eqs_carpda":{},
                    "road_eqs_carmot":{},
                     "cars":{},
                     "rail_pkm":{},
                     "bop121" : {}, 
                     "bop142" : {},
                     "bop143_148" : {},
                   }
    
    # In order to merge the data extracted from the data sets with the mobility frame, the function "merge" has to
    # be provided from both sides the columns it has to try to match. For instance, for "road_eqs_carage", the function merge will have to match
    # the values in mobility column "id_1" with those in column "age_type" in the data extracted from "road_eqs_carage".
     
    
    sort_dict = {"road_eqs_carage":{"id_1":"age_type", "year" : "year", "geo\\time" : "geo\\time"},
                  "road_eqs_mopeds":{"id_1":'vehicle', "year" : "year", "geo\\time" : "geo\\time"},
                    "road_eqs_motorc":{"id_1":"engine", "year" : "year", "geo\\time" : "geo\\time"},
                     "bus_coach":{"vehicle_type":"vehicle", "year" : "year", "geo\\time" : "geo\\time"},
                    "road_eqs_busmot":{"id_1":"mot_nrg", "year" : "year", "geo\\time" : "geo\\time"},
                    "rail_tf_traveh":{"id_1":'vehicle', "year" : "year", "geo\\time" : "geo\\time"},
                    "road_tf_road":{"id_2":'tra_infr', "year" : "year", "geo\\time" : "geo\\time"},
                    "avia_panc":{"id_1":"tra_meas", "type" : "flight_type", "year" : "year", "geo\\time" : "geo\\time"},
                    "avia_paincc":{"id_1":"tra_meas", "type" : "flight_type", "year" : "year", "geo\\time" : "geo\\time"},
                    "avia_paexcc":{"id_1":"tra_meas", "type" : "flight_type", "year" : "year", "geo\\time" : "geo\\time"},
                    "road_pa_mov":{"id_2":'vehicle', "year" : "year", "geo\\time" : "geo\\time"},
                    "road_eqs_carpda":{"id_3":'mot_nrg', "year" : "year", "geo\\time" : "geo\\time"},
                    "road_eqs_carmot":{"id_2":'engine', "id_3":'mot_nrg', "year" : "year", "geo\\time" : "geo\\time"},
                     "cars":{"vehicle_type" : "vehicle", "year" : "year", "geo\\time" : "geo\\time"},
                     "rail_pkm":{"vehicle_type" : "vehicle", "year" : "year", "geo\\time" : "geo\\time"},
                     "bop121" : {"type" : "flight_type", "year" : "year", "geo\\time" : "geo\\time"}, 
                     "bop142" : {"vehicle_type" : "vehicle", "year" : "year", "geo\\time" : "geo\\time"},
                     "bop143_148" : {"vehicle_type" : "vehicle", "year" : "year", "geo\\time" : "geo\\time"},

                }
    
    # Columns to be added to the data frames extracted from the different data sets.
    
    add_col_dict = {"road_eqs_carage":{},
                  "road_eqs_mopeds":{},
                    "road_eqs_motorc":{},
                    "road_eqs_busmot":{},
                    "rail_tf_traveh":{},
                    "road_tf_road":{},
                    "avia_panc":{"flight_type" : "national"},
                    "avia_paincc":{"flight_type" : "intra_eu"},
                    "avia_paexcc":{"flight_type" : "extra_eu"},
                    "road_pa_mov":{},
                    "road_eqs_carpda":{},
                    "road_eqs_carmot":{},
                    "cars":{},
                    "rail_pkm":{},
                    "bop121" : {},
                    "bop142" : {},
                    "bop143_148" : {}}
    
    # The factor the information retrieved from each of the data sets has to be multiplied for.
    
    units_dict = {"road_eqs_carage":1,
                  "road_eqs_mopeds":1,
                    "road_eqs_motorc":1,
                    "road_eqs_busmot":1,
                    "bus_coach":1000000000,
                    "rail_tf_traveh":0.001,
                    "road_tf_road":1,
                    "avia_panc":1,
                    "avia_paincc":1,
                    "avia_paexcc":1,
                    "road_pa_mov":1000000,
                    "road_eqs_carpda":1,
                    "road_eqs_carmot":1,
                    "cars":1000000000,
                    "rail_pkm":1000000000,
                    "bop121" : 1,
                    "bop142" : 1,
                    "bop143_148" : 1
                 }
    
    # The names of the different columns have been shortened, the correspondency would be as follows:
    # Number of vehicles : num_vehi, Share of number of vehicles per category : num_vehi_share, Millions of trains : trkm, 
    # Share of trains km per each category : trkm_share, Number of passengers : num_p, Km per flight : flight_km,
    # Contribution to passenger km : contr_pkm, Million passenger km : mill_pkm, Coeff. type of fuel : fuel_coeff, 
    # Coeff.engine displacement : eng_coeff, Coeff.technology : tech_coeff, Occupancy factor : ocp_factor.
    
    col_name_dict = {"road_eqs_carage" : "tech_coeff",
                     "road_eqs_carmot" : "eng_coeff",
                     "road_eqs_carpda" : "fuel_coeff",
                     "road_pa_mov" : "mill_pkm",
                     "bus_coach" : "mill_pkm",
                     "cars" : "mill_pkm",
                     "rail_pkm" : "mill_pkm",
                     "bop143_148" : "ocp_factor",
                     "bop142" : "contr_pkm",
                     "bop121" : "flight_km",
                     "avia_panc" : "num_p",
                     "avia_paincc" : "num_p",
                     "avia_paexcc" : "num_p",
                     "rail_tf_traveh" : "trkm",
                     "road_eqs_busmot" : "num_vehi",
                     "road_tf_road" : "mill_vehikm",
                     "road_eqs_motorc" : "num_vehi",
                     "road_eqs_mopeds" : "num_vehi"}
                     
                 
    # Classification of the different data sets according to their source.
    
    source_dict = {"road_eqs_carage" : "ESTAT",
                     "road_eqs_carmot" : "ESTAT",
                     "road_eqs_carpda" : "ESTAT",
                     "road_pa_mov" : "ESTAT",
                     "bus_coach" : "POCKETBOOK",
                     "cars" : "POCKETBOOK",
                     "rail_pkm" : "POCKETBOOK",
                     "bop143_148" : "BOP",
                     "bop142" : "BOP",
                     "bop121" : "BOP",
                     "avia_panc" : "ESTAT",
                     "avia_paincc" : "ESTAT",
                     "avia_paexcc" : "ESTAT",
                     "rail_tf_traveh" : "ESTAT",
                     "road_eqs_busmot" : "ESTAT",
                     "road_tf_road" : "ESTAT",
                     "road_eqs_motorc" : "ESTAT",
                     "road_eqs_mopeds" : "ESTAT"}
                    
                  
                 
    # The correspondency between the columns in the final mobility table with the data sets the information displayed in them comes from.
    # For instance, the information that will be displayed in column "tech_coeff" will come from "road_eqs_carage".
    
    merge_dict = {"tech_coeff" : ["road_eqs_carage"],
                  "eng_coeff" : ["road_eqs_carmot"],
                  "fuel_coeff" : ["road_eqs_carpda"],
                  "mill_pkm" : ["road_pa_mov", "cars", "rail_pkm", "bus_coach"],
                  "ocp_factor" : ["bop143_148"],
                  "contr_pkm" : ["bop142"],
                  "flight_km" : ["bop121"], 
                  "num_p" : ["avia_panc", "avia_paincc", "avia_paexcc"],
                  "trkm" : ["rail_tf_traveh"],
                  "num_vehi" : ["road_eqs_busmot", "road_eqs_motorc", "road_eqs_mopeds"],
                  "mill_vehikm" : ["road_tf_road"]}
    
    # In some cases, some categories/columns of some data sets require the addition of new elements. 

    extra_types = {"road_eqs_busmot" : {"mot_nrg" :["PET;LPG;DIE;ELC;ALT;DIE_X_HYB;ELC_DIE_HYB;ELC_DIE_PI;HYD_FCELL;OTH", "CNG;LNG"]},
                  "road_tf_road" : {'tra_infr' : ["RD_INB;RD_OUTB;RD_OTH"]},
                  "road_eqs_carage" : {},
                  "road_eqs_carmot" : {},
                   "road_eqs_carpda" : {},
                     "road_pa_mov" : {},
                     "cars" : {},
                     "rail_pkm" : {},
                     "bop143_148" : {},
                     "bop142" : {},
                     "bop121" : {},
                     "avia_panc" : {},
                     "avia_paincc" : {},
                     "avia_paexcc" : {},
                     "rail_tf_traveh" : {"vehicle" : ["RCA_ELC;LOC_ELC", "RCA_DIE;LOC_DIE"]},
                     "road_eqs_motorc" : {},
                     "road_eqs_mopeds" : {}}
                 

    # Therse are the names to be provided as argument to the function in order to obtain each of the dictionaries.
    
    col_dicts = {"FILTERS" : filters_dict, "SORT" : sort_dict, "UNITS" : units_dict, "COLNAMES" : col_name_dict,
                 "MERGE" : merge_dict, "SOURCE" : source_dict, "EXTRA" : extra_types, "ADDCOL" : add_col_dict}
    
    if (col_name == "MERGE"):
        
        return(col_dicts[col_name])
    
    else:
    
        return(col_dicts[col_name][DS])


In [7]:
def bop_report(bop_sheet, col_name):
    
    """
    Provide the information contained in the tables on pag.143,148,121 and 142 of the BoP Mobility report.
    
    :param bop_sheet : BoP mobility report sheet name.
    :type bop_sheet : str.
    :param col_name : reference to the information required within the sheet.
    :type col_name : str.
    :return : information required.
    :rtype : list/dict.
    
    """
    
    
    bop_table_dict = {"bop143_148" : {"vehicle" : ["Passenger_Cars", "2W", "Buses"], "value" : [1.62, 1.1, 14.7]},
                     "bop121" : {"flight_type" : ["national", "intra_eu", "extra_eu"], "value" : [np.nan ,1188, 6287]},
                      "bop142" : {"vehicle" : ["2W"], "value" : [0.026]}
                     }
    
    bop121_national ={'AT':245, 'BE':148, 'BG':282, 'CY':81, 'CZ':238, 'DE':506, 'DK':175, 'EE':180, 'EL':307, 'ES':1804, 'FI':492,
 'FR':677, 'HU':258, 'IE':246, 'IT':465, 'LT':216,'LU':43, 'LV':215, 'MT':50, 'NL':172, 'PL':473, "UK":418, 'PT':899, 'RO':413, 'SE':568,
                      'SI':120, 'SK':187}
    
    # Therse are the names to be provided as argument to the function in order to obtain each of the dictionaries.
    
    col_dicts = {"BOP_TABLE" : bop_table_dict, "BOP_NAT_FLIGHTS" : bop121_national}
    
    if col_name == "BOP_NAT_FLIGHTS":
        
        return(col_dicts[col_name])
    
    else:
    
        return(col_dicts[col_name][bop_sheet])
    

In [8]:
def check_countries(countries_names):
    
    """
    Check whether the country names are in a given dict, whether they are repeated and turn the country names into country codes.
    
    :param countries_names : countries names.
    :type countries_names : list.
    :return : countries in the provided list that belong to the given dict and that are not repeated.
    :rtype : list.

    
    """
    
    countries_dict = {'AT': 'Austria', 'BE': 'Belgium', 'BG': 'Bulgaria',
 'CY': 'Cyprus', 'CZ': 'Czechia', 'DE': 'Germany', 'DK': 'Denmark', 'EE': 'Estonia', 'EL': 'Greece', 'ES': 'Spain', 'FI': 'Finland',
 'FR': 'France', 'HR': 'Croatia', 'HU': 'Hungary', 'IE': 'Ireland', 'IT': 'Italy', 'LT': 'Lithuania','LU': 'Luxemburg', 'LV': 'Latvia',
    'MT': 'Malta', 'NL': 'Netherlands', 'PL': 'Poland', "UK" : "United Kingdom", 'PT': 'Portugal', 'RO': 'Romania', 'SE': 'Sweden', 'SI': 'Slovenia', 'SK': 'Slovakia',
               'LI' : 'Liechtenstein', "TR" : "Turkey", "XK" : "Kosovo", "NO" : "Norway", "MK" : "North Macedonia", "CH" : "Switzerland", "EU-28" : "EUROPEAN UNION (28)"
               , "ME" : "Montenegro", "IS" : "Iceland", "RS" : "Serbia", "AL" : "Albania", "TOTAL" : "Total"}
                
    countries_list = list(countries_dict.values())
    
    countries_dict_inverse = {v: k for k, v in countries_dict.items()}
    
    countries_repeated = predw.check_repeated(countries_names)
    
    countries_not_in_ds = predw.check_belong(countries_names, countries_list)
    
    countries_codes = [countries_dict_inverse[c] for c in countries_names if c not in countries_not_in_ds]
    
    return(countries_codes)
    

In [9]:
def add_col(input_df, DS):
    
    """
    Add a column to the data frame containing the information extracted from a data set.
    
    :param input_df : data extracted from a data set.
    :type input_df : data frame.
    :param DS : data set the information in input_df has been extracted from.
    :type DS : str.
    :return :  data originally extracted plus the information in the new column.
    :rtype : data frame.
    
    """
    
    add_col_dict = ds_transport(DS, "ADDCOL")
    
    if add_col_dict == {}:
        
        return(input_df)
    
    else:
        
        col = list(add_col_dict.keys())[0]
        
        value = add_col_dict[col]
        
        input_df[col] = value
        
        return(input_df)

In [10]:
def add_extra_type(input_df, DS):
    
    """
    Add new categories/types/elements in a column to the given data frame containing the information extracted from a data set.
    
    :param input_df : data extracted from a data set.
    :type input_df : data frame.
    :param DS : data set the information in input_df has been extracted from.
    :type DS : str.
    :return :  data originally extracted plus the new categories/types/elements in a column.
    :rtype : data frame.
    
    """
    

    if ds_transport(DS, "EXTRA") == {} or input_df.empty :
        
        return(input_df)
    
    else:
        
        extra_type_dict = ds_transport(DS, "EXTRA")
        
        ext_type_col = list(extra_type_dict.keys())[0]
        
        types_in_col = list(set(input_df[ext_type_col].tolist()))
        
        extra_types = extra_type_dict[ext_type_col]
        
        values_col = ds_transport(DS, "COLNAMES")
        
        possible_values = list(set(input_df[ext_type_col].tolist())) + extra_types
        
        input_df = input_df.pivot_table(values = values_col, index =  ["year", "geo\\time", "unit"], columns = ext_type_col, dropna = False).reset_index()
        
        for e_type in extra_types:
        
            e_types_list = e_type.split(";")
                     
            e_types_list = [t for t in e_types_list if t in types_in_col]
        
            input_df[e_type] = input_df[e_types_list].sum(axis = 1)
            
        id_cols = [col for col in input_df.columns if col not in possible_values]
            
        input_df = pd.melt(input_df, id_vars = id_cols, value_vars = possible_values)
        
        input_df.rename(columns = {"value" : values_col}, inplace  = True)
        
        return(input_df)

In [11]:
def get_mobility(start_year, end_year, countries_names, DS):
    
    """Downloads the data from the provided EUROSTAT data set, filters it for the countries and period given and fills up
    information gaps.
    
    :param start_year : first year of the period of interest.
    :type start_year : str.
    :param end_year : last year of the period of interest.
    :type end_year : str.
    :param countries_names : names of the countries of interest.
    :type countries : list.
    :param DS : EUROSTAT data set from which the information is downloaded.
    :type DS : str.
    :return: data frame containing the information required.
    :rtype: data frame.
    
    """
    
    data = estat.get_data_df(DS, flags=True)

    id_cols = [col for col in data.columns if not col.startswith(("1","2"))] # Identity columns.

    new_columns = [col.replace("_value", "") for col in data.columns]
    
    new_columns_dict = dict(zip(data.columns, new_columns))
    
    data = data.rename(columns = new_columns_dict)
    
    data.columns = [str(c) for c in data.columns]
    
    # Filter other categories.
    
    filters_dict = ds_transport(DS, "FILTERS")
    
    if filters_dict != {}:
        
        for key in filters_dict.keys():
            
            data = data[data[key].isin(filters_dict[key])]
    
    # Remove the rows with all the cells empty.
    
    data = data.set_index("geo\\time")
    
    data = data.dropna(axis = 0, how = "all")
    
    data = data.reset_index()
    
    # If the data for a particular year is missing it will be extrapolated out of the existing data through a linear regression.
    
    years = [str(y) for y in list(range(int(start_year), (int(end_year) + 1)))] 
    
    existing_years = [c for c in data.columns if c.isnumeric()]
    
    years_cols = [yr for yr in years if yr in data.columns]
    
    def extrapolate(df, nyr):
        df = df.dropna()
        y = df.tolist()
        if len(y)<2:
            return(np.nan)
        else:
            x = df.index.astype(float)
            f = interpolate.interp1d(x,y)
            # The new elements are chosen to make sure lie within the range of x values.
            new_x = [x[0], (x[0]+x[1])/2]
            new_y = f(new_x)
            m = np.diff(new_y)
            b = (sum(new_y)-(m*sum(new_x)))/2
            value = m*nyr+b
            # The extrapolation may also produce undesired negative values.
            if value[0]>0:
                return(value[0])
            else:
                return(np.nan)
    
    
    if len(years_cols) < len(years):
        
        missing_years = [yr for yr in years if yr not in years_cols]
        
        print("{} not in dataset {}. The values will be extrapolated".format(missing_years, DS))
            
        for new_yr in missing_years:
        
            data[new_yr] = data[existing_years].apply(lambda x: extrapolate(x, int(new_yr)),axis = 1)
    
    new_cols = id_cols + years
    
    data = data[new_cols]

    # If a country is missing from the data set, a new line will be added for that country with the values for the selected
    # years being the average of the corresponding values of the existing countries.
    
    countries_codes = check_countries(countries_names)
    
    absent_cc = [cc for cc in countries_codes if cc not in data["geo\\time"].tolist()]
    
    group_cols = [c for c in id_cols if c != "geo\\time"]
    
    data_gp = data.groupby(group_cols)
    
    data_mean = data.groupby(group_cols)[years].mean().reset_index()
    
    groups = data_gp.groups
    
    if len(absent_cc)>0:
        
        print("{} data is missing. The countries average will be used to fill the gap".format(absent_cc))
    
        for cc in absent_cc:
        
            data_mean["geo\\time"] = cc
            
            data = data.append(data_mean)
            
    # If for one country some information is missing (e.g. in "road_tf_road") Bulgaria does not have values for "tra_infr" = MWAY but it
    # does for other "tra_infr" values. The corresponding all countries mean value for the particular informartion missing is calculated
    # and introduce in the data frame in a new line with "geo\\time" column equal to the country of interest.

    for g in groups:
        
        data_g = data_gp.get_group(g)
        
        missing_cc = [ccode for ccode in countries_codes if ccode not in data_g["geo\\time"].tolist()]
        
        for ccode in missing_cc:

            data_cc = data_g.groupby(group_cols).mean().reset_index()

            data_cc["geo\\time"] = ccode
            
            data = data.append(data_cc)
            
    
    # Fills out nan values in the years columns with the column mean.

    data = data.fillna(data.mean())
     
    # Filters by country.
    
    data = data[data["geo\\time"].isin(countries_codes)]
    
    # Reorganize data frame.

    data = pd.melt(data, id_vars = id_cols, value_vars = years)

    # Adapt to right units.
    
    units_factor = ds_transport(DS, "UNITS")
    
    data["value"] = data["value"] * units_factor
    
    # Set columns names and data types.
    
    values_col = ds_transport(DS, "COLNAMES")
    
    data = data.rename(columns = {"variable" : "year", "value" : values_col})
    
    data.columns = [str(c) for c in data.columns]
    
    data["year"] = data["year"].astype(str)
    
    # Add extra types.
    
    data = add_extra_type(data, DS)
    
    # Add columns.
    
    data = add_col(data, DS)
    
    data = data.replace(0, np.nan)
          
    return(data)

In [12]:
def get_bop(start_year, end_year, countries_names, bop_pag):
    
    """
    Extract the information from the table in the BoP Mobility sheet provided for the years and countries given.
    
    :param start_year : first year of the period of interest.
    :type start_year : str.
    :param end_year : last year of the period of interest.
    :type end_year : str.
    :param countries_names : names of the countries of interest.
    :type countries : list.
    :param bop_pag : BoP Mobility sheet containing the table with the information.
    :type bop_pag : str.
    :return: data frame containing the information required.
    :rtype: data frame.
    
    NOTE: Since the information in the BoP Mobility report tables is the same for every year, here the start and end years
    will be use to add a column "year" to the resulting data frame instead of for filtering information.
    
    """
    
    years = [str(yr) for yr in list(range(int(start_year), int(end_year) +1 ))]
    
    countries_codes = check_countries(countries_names)
    
    value_col = ds_transport(bop_pag, "COLNAMES")
    
    bop_dict = bop_report(bop_pag, "BOP_TABLE")
    
    bop_nat_flights = bop_report("", "BOP_NAT_FLIGHTS")
    
    all_c_yr = []

    for yr in years:
        
        for c in countries_codes:
            
            data = pd.DataFrame(bop_dict)
            
            data["geo\\time"] = c
            
            data["year"] = yr
            
            data.loc[data["value"].isna(), "value"] = bop_nat_flights[c]
            
            all_c_yr.append(data)
            
    bop_df = pd.concat(all_c_yr)
    
    units_factor = ds_transport(bop_pag, "UNITS")
    
    bop_df["value"] = bop_df["value"] * units_factor
    
    bop_df = bop_df.rename(columns = {"value" : ds_transport(bop_pag, "COLNAMES")})
    
    return(bop_df)
       

In [13]:
def get_pocketbook(start_year, end_year, countries_names, file_name, file_folder, sheetname):
    
    """
    Read and filter information from an excel file containing the Eurostat Pocketbook data, filters it for the countries and years chosen
    and fills up information gaps.
    
    :param start_year : first year of the period of interest.
    :type start_year : str.
    :param end_year : last year of the period of interest.
    :type end_year : str.
    :param countries_names : names of the countries of interest.
    :type countries : list.
    :param file_name : name of the excel file containing the information.
    :type file_name : str.
    :param file_fodler : path to the folder where the excel file is located.
    :type file_folder : str.
    :param sheetname : name of the sheet within the excel file where the information is located.
    :type sheetname : str/int(position of the sheet beginning with 0).
    :return : the pocketbook data filtered according to the input parameters.
    :rtype : data frame.
    
    """
    
    data = di.read_file(file_name, file_folder, 4, sheetname, {})
 
    data.rename(columns = {data.columns[0] : "geo\\time"}, inplace = True)
    
    # There are some rows with no figures but only "-" in each cell. These rows will be delted. The same will be doen to the rows
    # with cell "geo\\time" empty.
    
    data = data[~data["geo\\time"].isna()]
    
    data = data.replace({"-" : np.nan, " " : np.nan})
    
    data = data.set_index("geo\\time")
    
    data = data.dropna(axis = 0, how = "all")
    
    data = data.reset_index()
    
    data.columns = [str(c) for c in data.columns]
    
    # If the data for a particular year is missing it will be extrapolated out of the existing data through a linear regression.
    
    existing_years = [c for c in data.columns if c.isnumeric()]
    
    years = [str(y) for y in list(range(int(start_year), (int(end_year) + 1)))] 
    
    years_cols = [yr for yr in years if yr in data.columns]
    
    def extrapolate(df, nyr):
        df = df.dropna()
        y = df.tolist()
 
        if len(y)<2:
            return(np.nan)
        else:
            x = df.index.astype(float)
            f = interpolate.interp1d(x,y)
            # The new elements are chosen to make sure lie within the range of x values.
            new_x = [x[0], (x[0]+x[1])/2]
            new_y = f(new_x)
            m = np.diff(new_y)
            b = (sum(new_y)-(m*sum(new_x)))/2
            value = m*nyr+b
            # The extrapolation may also produce undesired negative values.
            if value[0]>0:
                return(value[0])
            else:
                return(np.nan)
    
    if len(years_cols) < len(years):
        
        missing_years = [yr for yr in years if yr not in years_cols]
        
        print("{} not in dataset. The values will be extrapolated.".format(missing_years))

        for new_yr in missing_years:

            data[new_yr] = data[existing_years].apply(lambda x: extrapolate(x, int(new_yr)),axis = 1)

    # Filter by year.
        
    data = data[["geo\\time"] + years]
                                      
    # If a country is missing from the data set, a new line will ba added for that country with the values for the selected
    # years being the average of the corresponding values of the existing countries.
             
    countries_codes = check_countries(countries_names)
    
    absent_cc = [cc for cc in countries_codes if cc not in data["geo\\time"].tolist()] 
    
    data = data.set_index("geo\\time")                              
    
    if len(absent_cc)>0:
        
        print("{} data is missing. The countries average will be used to fill the gap".format(absent_cc))
    
        for cc in absent_cc:
        
            data.loc[cc] = df.mean()
            
    data = data.reset_index()
                                      
    # Fills out nan values in the years columns with the column mean.

    data = data.fillna(data.mean())

    # Filter by country.
    
    data = data[data["geo\\time"].isin(countries_codes)]
    
    # Reorganize data frame.
    
    data = pd.melt(data, id_vars = ["geo\\time"], value_vars = years_cols)
    
    # Add vehicle column.
    
    vehicle_dict = {"cars" : "Passenger_Cars", "rail_pkm" : "Trains", "bus_coach" : "Buses", "motorway" : "Buses", "CNLPNL" : "Buses"}
    
    data["vehicle"] = vehicle_dict[sheetname]
    
    # Adapt to right units.
    
    units_factor = ds_transport(sheetname, "UNITS")
    
    data["value"] = data["value"] * units_factor
    
    # Rename and change columns data type.
    
    data = data.rename(columns = {"variable" : "year", "value" : ds_transport(sheetname, "COLNAMES")})
    
    data["year"] = data["year"].astype(str)
    
    return(data)


In [14]:
def calc_fuel_coeff(start_year, end_year, countries_names):
    
    """
    Extract the required information and calculate the type of fuel coefficient for the provided years and countries.
    
    :param start_year : first year of the period of interest.
    :type start_year : str.
    :param end_year : last year of the period of interest.
    :type end_year : str.
    :param countries_names : names of the countries of interest.
    :type countries : list.
    :return : fuel coefficient for the provided years and countries.
    :rtype : data frame.
    
    """
    
    DS = "road_eqs_carpda"
    
    value_col = ds_transport(DS, "COLNAMES")
    
    data = get_mobility(start_year, end_year, countries_names, DS)

    data_grp = data.groupby(["geo\\time", "year"])
    
    groups = data_grp.groups
    
    all_groups = []
    
    for g in groups:
        
        df = data_grp.get_group(g)
        
        total = df[df["mot_nrg"] == "TOTAL"][value_col].tolist()[0]
        
        if math.isnan(total):
            
            total = df[df["mot_nrg"] != "TOTAL"][value_col].sum()
        
        df.loc[:, value_col] = df.loc[:, value_col].div(total)
        
        all_groups.append(df)
        
    new_df = pd.concat(all_groups)
    
    return(new_df)
    
    

In [15]:
def calc_eng_coeff(start_year, end_year, countries_names):
    
    """
    Extract the required information and calculate the engine displacement coefficient for the provided years and countries.
    
    :param start_year : first year of the period of interest.
    :type start_year : str.
    :param end_year : last year of the period of interest.
    :type end_year : str.
    :param countries_names : names of the countries of interest.
    :type countries : list.
    :return : engine displacement coefficient for the provided years and countries.
    :rtype : data frame.
    
    """
    
    DS = "road_eqs_carmot"
    
    value_col = ds_transport(DS, "COLNAMES")
    
    data = get_mobility(start_year, end_year, countries_names, DS)

    data_grp = data.groupby(["geo\\time", "year", "mot_nrg"])
    
    groups = data_grp.groups
    
    all_groups = []
    
    for g in groups:
        
        df = data_grp.get_group(g)
        
        total = df.loc[df["engine"] == "TOTAL"][value_col].tolist()[0]
        
        if math.isnan(total):
            
            total = df.loc[df["engine"] != "TOTAL"][value_col].sum()
        
        df.loc[:, value_col] = df.loc[:, value_col].div(total)
        
        all_groups.append(df)
        
    new_df = pd.concat(all_groups)
    
    return(new_df)

In [16]:
def calc_tech_coeff(start_year, end_year, countries_names):
    
    """
    Extract the required information and calculate the technology coefficient for the provided years and countries.
    
    :param start_year : first year of the period of interest.
    :type start_year : str.
    :param end_year : last year of the period of interest.
    :type end_year : str.
    :param countries_names : names of the countries of interest.
    :type countries : list.
    :return : technology coefficient for the provided years and countries.
    :rtype : data frame.
    
    """
    # The car definition according to the year of production. Take into account that "range" does not consider the last value
    # of the interval, i.e. list(range(1980,1992)) last value is 1991.
    
    age_type_dict = {"CONV" : list(range(1980,1992)), 
                "EU1" : list(range(1992,1996)),
                "EU2" : list(range(1996,2000)),
                "EU3" : list(range(2000,2005)),
                "EU4" : list(range(2005,2010)),
                "EU5" : list(range(2010,2015)),
                "EU6" : list(range(2015,2021)),
               }
    
    DS = "road_eqs_carage"
    
    value_col = ds_transport(DS, "COLNAMES")
    
    data = get_mobility(start_year, end_year, countries_names, DS)
    
    data["year"] = data["year"].astype(int)
    
    def calc_age_years(x):
        
        ref_yr = x["year"]
        
        if x["age"] == 'Y_LT2':
            
            years = list(np.array(range(ref_yr-2,ref_yr)) +1)
            
        elif x["age"] == 'Y2-5':
            
            years = list(np.array(range(ref_yr-5,ref_yr-2)) +1)
            
        elif x["age"] == 'Y5-10':
            
            years = list(np.array(range(ref_yr-10,ref_yr-5)) +1)
            
        elif x["age"] == 'Y10-20':
            
            years = list(np.array(range(ref_yr-20,ref_yr-10)) +1)
            
        elif x["age"] == 'Y_GT10':
            
            years = list(np.array(range(ref_yr-20,ref_yr-10)) +1)
            
        elif x["age"] == 'Y_GT20':
            
            years = list(np.array(range(1979,ref_yr-20)) +1)
            
        else:
            
            years = list(range(1980, ref_yr + 1))
            
        return(years)
    
    def calc_share(age_years, age_type):
        
        in_years = [yr for yr in age_years if yr in age_type_dict[age_type]]
        
        sh = len(in_years) / len(age_years)
        
        return(sh)    
    
    data_grp = data.groupby(["geo\\time"])
    
    groups = data_grp.groups
    
    all_groups = []
    
    for g in groups:
        
        df = data_grp.get_group(g)
        
        df["age_years"] = df.apply(lambda x : calc_age_years(x), axis  = 1)
        
        for age_type in age_type_dict.keys():
            
            share_col = "_".join([age_type, "share"])
            
            df[share_col] = df["age_years"].apply(lambda x : calc_share(x, age_type))
            
            df[age_type] = df[value_col] * df[share_col]
             
        final_df = df[df["age"]!="TOTAL"].groupby(["geo\\time", "year"]).sum()
        
        final_df.reset_index(inplace = True)
    
        all_groups.append(final_df)
        
    new_df = pd.concat(all_groups)
    
    # Add extra age types.
    
    extra_age_types = ["CONV;EU1;EU2;EU3", "CONV;EU1;EU2;EU3;EU4;EU5", "CONV;EU1;EU2;EU3;EU4;EU5;EU6"]
    
    for e_age_type in extra_age_types:
        
        e_age_types_list = e_age_type.split(";")
        
        new_df[e_age_type] = new_df[e_age_types_list].sum(axis = 1)
    
    new_df = pd.melt(new_df, id_vars = ["geo\\time", "year"], value_vars = list(age_type_dict.keys()) + extra_age_types)
    
    # Calculate the % of each age type over the total per country and year.
    
    total_df = new_df[new_df["variable"] == "CONV;EU1;EU2;EU3;EU4;EU5;EU6"]
    
    total_df = total_df.rename(columns = {"value" : "total_value"})
    
    new_df = new_df.rename(columns = {"value" : value_col, "variable" : "age_type"})
    
    final_df = new_df.merge(total_df[["geo\\time", "year", "total_value"]], how = "left", on = ["geo\\time", "year"])
    
    final_df["year"] = final_df["year"].astype(str)
    
    final_df[value_col] = final_df[value_col] / final_df["total_value"]
    
    return(final_df)

In [28]:
def mob_merge(mob_frame, col_name, start_year, end_year, countries_names, file_name, file_folder):
    
    """
    For the years and countries of interest, extracts the information from the data set/data source that should go 
    in the given column in the final data frame and merges it with the mobility frame.
    
    :param mob_frame : mobility frame.
    :type mob_frame : data frame.
    :param col_name : name of the column in the final data frame where the extracted information will be displayed.
    :type col_name : str.
    :param start_year : first year of the period of interest.
    :type start_year : str.
    :param end_year : last year of the period of interest.
    :type end_year : str.
    :param countries_names : names of the countries of interest.
    :type countries : list.
    :param file_name : the name of the file containing the information in the case the data source is POCKETBOOK.
    :type file_name : str.
    :param file_folder : the path to the folder containing he file with the information in the case the data source is POCKETBOOK.
    :type file_folder : str.
    :return : a mobility table containing the mobility frame plus one column (col_name).
    :rtype : data frame.
    
    """
    
    data_sets = ds_transport("", "MERGE")[col_name]
    
    years = [str(yr) for yr in list(range(int(start_year), int(end_year) +1 ))]
    
    countries_codes = check_countries(countries_names)
    
    mob_frame_all = []
    
    for yr in years:
        
        for cc in countries_codes:
            
            mf = mob_frame.copy()
            
            mf["geo\\time"] = cc
            
            mf["year"] = yr
            
            mob_frame_all.append(mf)
            
    new_mob_frame = pd.concat(mob_frame_all)
    
    m_cols = list(new_mob_frame.columns)
    
    all_merged = []

    for ds in data_sets:
        
        source = ds_transport(ds, "SOURCE")
        
        if source == "ESTAT":
            
            if ds == "road_eqs_carmot":
                
                data = calc_eng_coeff(start_year, end_year, countries_names)
                
            elif ds == "road_eqs_carpda":
                
                data = calc_fuel_coeff(start_year, end_year, countries_names)
                
            elif ds == "road_eqs_carage":
                
                data = calc_tech_coeff(start_year, end_year, countries_names)
            
            else:
            
                data = get_mobility(start_year, end_year, countries_names, ds)
            
        elif source == "POCKETBOOK":
            
            data = get_pocketbook(start_year, end_year, countries_names, file_name, file_folder, ds)
            
        else:
            
            data = get_bop(start_year, end_year, countries_names, ds)
        
        merge_dict = ds_transport(ds, "SORT")

        m = new_mob_frame.merge(data, how = "left", left_on = list(merge_dict.keys()), right_on = list(merge_dict.values()))
        
        new_cols = [c for c in m.columns if c in m_cols or c == col_name]
        
        m = m[new_cols]
        
        all_merged.append(m)
        
    if len(all_merged) == 1:
            
        return(all_merged[0])
        
    else:
            
        merged = all_merged[0]
            
        for i in range(1, len(all_merged)):
                
            merged[col_name] = merged[col_name].fillna(all_merged[i][col_name])
    
        return(merged)
        

In [18]:
def share_columns(input_df):
    
    """
    For the columns num_vehi and trkm in the mobility table, calculates, for two different vehicle types, 2W and Trains, 
    the share of each of the different classes within each of the vehicle types.
    
    :param input_df : mobility table.
    :type input_df : data frame.
    :return : mobility table plus two new columns containing the share values.
    :rtype : data frame.
    
    """

    data_gr = input_df.groupby(["geo\\time", "year"])

    groups = data_gr.groups
    
    def share_calc(x, col, total):
        
        if x["vehicle_type"] in col:
            
            if col == "2W":
            
                return(x["num_vehi"]/total)
            
            elif col == "Trains":
                
                return(x["trkm"]/total)
            
            elif col == "Buses_fuel":
                
                return(x["num_vehi"]/total)
            
            elif col == "Buses_road":
                
                return(x["mill_vehikm"]/total)
            
            else: 
                
                print("Non-recognized column")
        
        else:
            
            return(np.nan)

    all_groups = []


    for g in groups:
    
        df = data_gr.get_group(g)
        
        twow_total = df[df["vehicle_type"] == "2W"]["num_vehi"].sum()
        
        trains_total = df[df["vehicle_type"] == "Trains"]["trkm"].sum()
        
        bus_road_total = df[df["vehicle_type"] == "Buses"]["mill_vehikm"].sum()
        
        bus_fuel_total = df[df["vehicle_type"] == "Buses"]["num_vehi"].sum()
    
        df["num_vehi_share"] = df.apply(lambda x : share_calc(x, "2W", twow_total), axis = 1)

        df["trkm_share"] = df.apply(lambda x : share_calc(x, "Trains", trains_total), axis = 1)
        
        df["bus_fuel_share"] = df.apply(lambda x : share_calc(x, "Buses_fuel", bus_fuel_total), axis = 1)

        df["bus_road_share"] = df.apply(lambda x : share_calc(x, "Buses_road", bus_road_total), axis = 1)
        
        df[df["vehicle_type"]!="2W"]["num_vehi_share"] = np.nan
        
        df[df["vehicle_type"]!="Trains"]["trkm_share"] = np.nan
        
        df[df["vehicle_type"]!="Buses"]["bus_fuel_share"] = np.nan
        
        df[df["vehicle_type"]!="Buses"]["bus_road_share"] = np.nan

    
        all_groups.append(df)
    
    new_df = pd.concat(all_groups)
    
    return(new_df)
        
      

In [19]:
def consumption(input_df):
    
    colnames_list = ["num_vehi_share", "trkm_share", "bus_road_share", "bus_fuel_share", "num_p", "flight_km", "contr_pkm", "mill_pkm",  "eng_coeff", "tech_coeff", "fuel_coeff"]
    
    prod_df = input_df[colnames_list]
    
    input_df["Result"] = prod_df.product(axis  = 1)
    
    input_df["Result"] = input_df["Result"].div(input_df["ocp_factor"], fill_value = 1)
    
    return(input_df)

In [30]:
def mobility(mob_frame, start_year, end_year, countries_names, file_name, file_folder):
    
    """
    For the years and countries of interest, extracts the information from all the data set/data source, filters it, makes
    the required calculations and merge them with the mobility frame.
    
    :param mob_frame : mobility frame.
    :type mob_frame : data frame.
    :param start_year : first year of the period of interest.
    :type start_year : str.
    :param end_year : last year of the period of interest.
    :type end_year : str.
    :param countries_names : names of the countries of interest.
    :type countries : list.
    :param file_name : the name of the file containing the information in the case the data source is POCKETBOOK.
    :type file_name : str.
    :param file_folder : the path to the folder containing he file with the information in the case the data source is POCKETBOOK.
    :type file_folder : str.
    :return : a mobility table containing the mobility frame plus all the required columns.
    :rtype : data frame.
    
    """

    colnames_list = ["num_vehi", "trkm", "num_p", "flight_km", "contr_pkm", "mill_pkm", "mill_vehikm",  "eng_coeff", "tech_coeff", "fuel_coeff", "ocp_factor"]

    mob_frame_cols = list(mob_frame.columns)

    mob_frame_cols.append("geo\\time")

    mob_frame_cols.append("year")

    all_colnames = []

    for col in colnames_list:
    
        print(col)
    
        m = mob_merge(mob_frame, col, start_year, end_year, countries_names, file_name, file_folder)
    
        all_colnames.append(m)

    all_merged = []

    all_merged.append(all_colnames[0])
    
    for i in range(1, len(all_colnames)):
    
        previous = all_merged[i-1]
    
        all_merged.append(previous.merge(all_colnames[i], how = "left", on = mob_frame_cols))

    final_merged = all_merged[-1]
    
    final_merged_complete = share_columns(final_merged)
    
    # Filling the tech_coeff and eng_coeff values for the LPG car
    
    final_merged_complete.loc[final_merged_complete["type"] == "LPG", "eng_coeff"] = 1
    
    final_merged_complete.loc[final_merged_complete["type"] == "LPG", "tech_coeff"] = 1
    
    # Calculating the consumption (¿?)
    
    consumption_df = consumption(final_merged_complete)
    
    # Adding units.
    
    final_merged_complete["units"] = "VKM"
    
    final_merged_complete.loc[final_merged_complete["code"].isin(["SP23","SP24","SP25", "SP26", "SP27"]),"units"] = "PKM"
    
    return(final_merged_complete)